# Semana 4 - Michel B Fiorio

### Parte 1:

1 - Com base no que vimos no curso, aponte em quais casos as redes neurais recorrentes (RNN) e redes neurais convolucionais (CNN) devem ser aplicadas. Quais características dessas redes favorecem cada tipo de aplicação?

**Resposta**: 

As redes neurais convolucionais são redes neurais que utilizam da operação matemática da convolução em pelo menos uma de suas camadas. A inspiração para a criação das camadas convolucionais se dá na organização do córtex visual humano. Em se tratando da arquitetura da camada convolucional de uma CNN, cada neurônio dessa camada está conectado a um número limitado de entradas dos neurônios da camada subsequente. Devido a sua arquitetura as CNNs são especialmente utilizadas em tarefas complexas de classificação de imagens.

Já nas redes neurais recorrentes, o seu diferencial está na implementação de loops de realimentação da informação de saída de neurônios. Esses loops criam a capacidade de memória nos nós da rede e com isso vem a capacidade de noção de ordem das informações no tempo. A rede se torna capaz de interpretar informações que tenham sequência temporal e por isso ela é aplicada em problema onde os dados de entrada tem alguma dependência temporal, como series temporais de ações da bolsa de valores, textos, arquivos de áudio da fala humana, etc.

2 - No que consiste a técnica Transfer Learning? Por que essa técnica é muito utilizada em problemas relacionados com imagens?

**Resposta**:

A técnica de Transfer Learning consiste na reutilização de modelos já treinados para criação de novos modelos que executarão tarefas similares. Essa técnica permite minimizar o tempo e custo computacional no treinamento de modelos profundos. O Transfer Learning é útil e frequentemente utilizado em modelos de classificação de imagens, principalmente em aplicações onde a quantidade de dados disponível para treinamento do modelo é limitada tornando impossível o treinamento de um modelo a partir do zero.



3 - O que são os autoencoders? Explique com suas palavras como esse tipo de modelo funciona.

**Resposta**: 

Autoencoders são redes neurais que combinam a aplicação de duas redes feedforward para formar uma rede maior que ganha capacidades complexas que as diferenciam das demais, tais como a geração de dados novos e segmentação de imagens. A ideia se baseia no fato de que em uma rede neural normal as informações extraídas dos dados de entrada são armazenadas nas camadas e a cada camada a rede vai aumentando a complexidade da representação das características dos dados. Então, é acoplada uma outra rede à saída dessa primeira. Essa segunda rede ganha a função de "desenrolar" a representação dos dados feita na primeira rede para reconstruir o dado original. Portanto o autoencoder tem a característica de possuir o mesmo número de neurônios na entrada e na saída. A primeira rede chama-se encoder e a segunda rede é chamada decoder. A saída será uma representação esparsa dos dados de entrada.


4 - Procure algum exemplo de aplicação recente de técnicas de Aprendizado Profundo para a solução de algum problema que ache relevante (classificação/identificação de doenças em plantas, diagnóstico médico, anaĺise de textos em redes sociais, previsão de ações da bolsa de valores, etc.). Explique o exemplo encontrado, destacando a importância do problema, a estratégia usada para resolvê-lo e os resultados obtidos.

**Resposta**: 

O estudo disponível no link abaixo se propôs a apresentar um comparativo dos resultados de uma rede neural convolucional comparada com outras duas técnicas tradicionais na aplicação de detecção de pedestres (pessoas em pé) em imagens. Motivados pela grande importância e utilidade desse problema para aplicações de segurança, robótica e carros autônomos, dentre outras, diversos trabalhos tem sido realizados com proposições de arquiteturas específicas para essa tarefa.

Nesse estudo em particular foi realizado o comparativo dos resultados da CNN com as técnicas Features de Haar e Histograma de Gradientes Orientado (HOG). Entretanto, dado o interesse apenas nos resultados da CNN, excluiremos o comparativo entre as técnicas.

A CNN com o melhor resultado na tarefa de detecção de pedrestes foi obtida através de um transfer learning da rede AlexNet. A base de dados de treinamento foi a Caltech, uma base de dados popular de detecção de pedrestes. A base de testes possuia 21084 exemplos de pedestres e 22384 imagens sem pedestres. As métricas obtidas dos resultados foram a sensitividade, especificidade, precisão e acurácia. Segue abaixo os resultados obtidos para a CNN:

- Sensitividade: 98,94%
- Especificidade: 99,17%
- Precisão: 99,12%
- Acurácia: 95,98%

Referência bibliográfica: http://gibis.unifesp.br/sibgrapi16/eproceedings/wuw/7.pdf

### Parte 2:

Aplique a técnica de transfer learning para treinar um novo modelo para classificar o dataset CIFAR-10, usado na Atividade 2. Comente sobre os resultados obtidos neste experimento quando comparados com os resultados obtidos na realização da Atividade 2. Foi mais fácil e rápido treinar o modelo usando transfer learning?


DICA: O script pode uma modificação do que foi usado na Atividade 2, alterando apenas a parte de construção do modelo para usar um modelo pré-treinado (recomenda-se o VGG16) seguido de camadas totalmente conectadas, como no exemplo Transfer_learning_CNN.ipynb, disponibilizado na plataforma.


In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

#### Importanto dados

In [2]:
# carregando dataset

(treinamentoX, treinamentoY), (testeX, testeY) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [3]:
treinamentoX.shape

(50000, 32, 32, 3)

In [4]:
# Convertendo valores dos pixels em float
treinamentoX = treinamentoX.astype('float32')
testeX = testeX.astype('float32')

# normalização para escala [0-1]
treinamentoX = treinamentoX / 255.0
testeX = testeX / 255.0

# transformando a variável alvo (target) para uma codificação one hot
treinamentoY = to_categorical(treinamentoY)
testeY = to_categorical(testeY)

In [5]:
testeY[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32)

#### Defininido modelo obtido no exercício da semana 2 (sem Transfer Learning)

In [6]:
# definindo modelo
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))  #O 'input_shape' só precisa ser fornecido na 1ª camada convolucional
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(.2))
  model.add(Conv2D(64, (3, 3), activation='relu'))  
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(.2))
  model.add(Conv2D(64, (3, 3), activation='relu'))  
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(.2))
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  
  # compilando modelo
  opt = Adam(learning_rate=0.001)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

#### Treinando o modelo (sem Transfer Learning)

In [7]:
# criando o modelo
model = define_model()
  
# treinamento do modelo
history = model.fit(treinamentoX, treinamentoY, 
                    epochs=10, batch_size=32, 
                    validation_split=0.2, 
                    verbose=1)
  


Epoch 1/10
1250/1250 [==============================] - 76s 59ms/step - loss: 1.6990 - accuracy: 0.3719 - val_loss: 1.4011 - val_accuracy: 0.4998
Epoch 2/10
1250/1250 [==============================] - 75s 60ms/step - loss: 1.3466 - accuracy: 0.5168 - val_loss: 1.2160 - val_accuracy: 0.5750
Epoch 3/10
1250/1250 [==============================] - 79s 63ms/step - loss: 1.2211 - accuracy: 0.5659 - val_loss: 1.0947 - val_accuracy: 0.6161
Epoch 4/10
1250/1250 [==============================] - 73s 58ms/step - loss: 1.1388 - accuracy: 0.5985 - val_loss: 1.0301 - val_accuracy: 0.6432
Epoch 5/10
1250/1250 [==============================] - 72s 58ms/step - loss: 1.0817 - accuracy: 0.6209 - val_loss: 1.0663 - val_accuracy: 0.6353
Epoch 6/10
1250/1250 [==============================] - 73s 58ms/step - loss: 1.0390 - accuracy: 0.6347 - val_loss: 0.9234 - val_accuracy: 0.6813
Epoch 7/10
1250/1250 [==============================] - 75s 60ms/step - loss: 1.0051 - accuracy: 0.6432 - val_loss: 0.9262 -

In [8]:
# desempenho do modelo para os dados de teste
_, acc = model.evaluate(testeX, testeY, verbose=0)
print('Desempenho do modelo para os dados de teste > %.3f' % (acc * 100.0))

Desempenho do modelo para os dados de teste > 68.330


#### Defininido modelo a partir do Transfer Learning da VGG16

In [9]:
# definindo modelo usando transfer learning

# Carregamos o modelo VGG16 sem as últimas camadas totalmente conectadas
pre_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

# Aqui fazemos com que as camadas do modelo pré-treinado não sejam alteradas durante o treino
for layer in pre_model.layers:
  layer.trainable = False

# Criando o modelo sequencial
def define_transfer_model():
  model = Sequential()
  model.add(pre_model)
  model.add(Dropout(.2))
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  
  # compilando modelo
  opt = Adam(learning_rate=0.001)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

58889256/58889256 [==============================] - 1s 0us/step


#### Treinando o modelo com Transfer Learning

In [10]:
# criando o modelo
model = define_model()
  
# treinamento do modelo
history = model.fit(treinamentoX, treinamentoY, 
                    epochs=10, batch_size=32, 
                    validation_split=0.2, 
                    verbose=1)

Epoch 1/10
1250/1250 [==============================] - 75s 58ms/step - loss: 1.7025 - accuracy: 0.3678 - val_loss: 1.4218 - val_accuracy: 0.4780
Epoch 2/10
1250/1250 [==============================] - 73s 58ms/step - loss: 1.3874 - accuracy: 0.4985 - val_loss: 1.2523 - val_accuracy: 0.5457
Epoch 3/10
1250/1250 [==============================] - 71s 57ms/step - loss: 1.2614 - accuracy: 0.5498 - val_loss: 1.1234 - val_accuracy: 0.6078
Epoch 4/10
1250/1250 [==============================] - 77s 62ms/step - loss: 1.1792 - accuracy: 0.5774 - val_loss: 1.0622 - val_accuracy: 0.6339
Epoch 5/10
1250/1250 [==============================] - 78s 62ms/step - loss: 1.1121 - accuracy: 0.6058 - val_loss: 1.0203 - val_accuracy: 0.6472
Epoch 6/10
1250/1250 [==============================] - 74s 59ms/step - loss: 1.0727 - accuracy: 0.6196 - val_loss: 1.0250 - val_accuracy: 0.6407
Epoch 7/10
1250/1250 [==============================] - 72s 58ms/step - loss: 1.0328 - accuracy: 0.6354 - val_loss: 0.9651 -

In [11]:
# desempenho do modelo para os dados de teste
_, acc = model.evaluate(testeX, testeY, verbose=0)
print('Desempenho do modelo para os dados de teste > %.3f' % (acc * 100.0))

Desempenho do modelo para os dados de teste > 67.210


**Comentário dos resultados**:

- Tempo de treinamento do modelo da semana 2 (sem k-fold): 12 min
- Tempo de treinamento do modelo com Transfer Learning: 12 min

- Desempenho do modelo da semana 2: 68,33%
- Desempenho do modelo com Transfer Learning: 67,21%

Conforme podemos observamor nos resultados obtidos, não houve melhora significa do tempo de treinamento e no desempenho do modelo ao se aplicar a técnica de transfer learning nesse caso específico.